## Step 60: Generate SWC PREC Datasets

Purpose: combine SWC stations with CREAT3 projected data, historic extreme event data and rainfall 
distribution datasets.  Partition data by station into individual feature classes modelling the 
final text file outputs.


In [18]:
import arcpy,os;
from arcpy.sa import *;

print("Step 60: Generate SWC PREC Datasets");

fgdb    = os.getcwd() + os.sep + 'source.gdb';
rezfgdb = os.getcwd() + os.sep + 'results.gdb';
wrkfgdb = os.getcwd() + os.sep + 'working.gdb';

if not arcpy.Exists(rezfgdb):
    print("  Creating new results workspace.");   
    arcpy.CreateFileGDB_management(
         os.path.dirname(rezfgdb)
        ,os.path.basename(rezfgdb)
    );
else:
    print("  Using preexisting results workspace.");

wrkstat   = wrkfgdb + os.sep + 'D4EMLite_PREC_Details_WRK';
stat_cnt  = arcpy.GetCount_management(wrkstat)[0];

grid      = rezfgdb + os.sep + 'CRWU_CREAT_Grid_Projections_DATA_INTERP';
grid_cnt  = arcpy.GetCount_management(grid)[0];

hist      = fgdb + os.sep + 'CRWU_CREAT_Historic_Climate_Stations';
hist_cnt  = arcpy.GetCount_management(hist)[0];

rfcov     = rezfgdb + os.sep + 'RainfallDistribution';
rfcov_cnt = arcpy.GetCount_management(rfcov)[0];

print("  Station Count          : " + str(stat_cnt));
print("  Interpolated Grid Count: " + str(grid_cnt));
print("  Historic Station Count : " + str(hist_cnt));
print("  Rainfall Distribution Coverage Count: " + str(rfcov_cnt));


Step 60: Generate SWC PREC Datasets
  Using preexisting results workspace.
  Station Count          : 5144
  Interpolated Grid Count: 6332
  Historic Station Count : 11165
  Rainfall Distribution Coverage Count: 19614


### 60.10: Calculate nearest historical station

In [19]:
%%time

in_file  = wrkstat;
out_file = wrkfgdb + os.sep + 'D4EMLite_PREC_Details_WRK2';

if arcpy.Exists(out_file):
    arcpy.Delete_management(out_file);
   
print("  copying to new work stations");
arcpy.conversion.FeatureClassToFeatureClass(
     in_features = in_file
    ,out_path    = os.path.dirname(out_file)
    ,out_name    = os.path.basename(out_file)
);

print("  add tracking fields for joined data");
arcpy.management.AddFields(
     in_table = out_file
    ,field_description = [
         ['has_creat3_grid_data'     ,'TEXT','Has CREAT3 Grid Data', 1]
        ,['is_interp_grid_data'      ,'TEXT','Is Interpolated Grid Data', 1]
        ,['has_creat3_histstorm_data','TEXT','Has CREAT3 Historic Storm Station Data',1]
        ,['has_rainfall_data'        ,'TEXT','Has Rainfall Data',1]
    ]
);

print("  calculating the nearest historic station");
arcpy.analysis.Near(
     in_features   = out_file
    ,near_features = hist
    ,search_radius = '15 degree'
    ,location      = 'NO_LOCATION'
    ,method        = 'GEODESIC'
);

print("  updating Near fields for clarity");
arcpy.management.AlterField(
     in_table        = out_file
    ,field           = 'NEAR_FID'
    ,new_field_name  = 'NEAR_FID_HIST'
    ,new_field_alias = 'NEAR_FID_HIST'
);

arcpy.management.AlterField(
     in_table        = out_file
    ,field           = 'NEAR_DIST'
    ,new_field_name  = 'NEAR_DIST_HIST'
    ,new_field_alias = 'NEAR_DIST_HIST'
);

print("  adding indexes");
arcpy.management.AddIndex(
     in_table        = out_file
    ,fields          = 'NEAR_FID_HIST'
    ,index_name      = 'NEAR_FID_HIST_IDX'
);

flds = [
     'IntensePrecip_Historic_5Year_24'
    ,'IntensePrecip_Historic_10Year_2'
    ,'IntensePrecip_Historic_15Year_2'
    ,'IntensePrecip_Historic_30Year_2'
    ,'IntensePrecip_Historic_50Year_2'
    ,'IntensePrecip_Historic_100Year_'
];

print("  joining in historic storms information");
arcpy.management.JoinField(
     in_data         = out_file
    ,in_field        = 'NEAR_FID_HIST'
    ,join_table      = hist
    ,join_field      = 'OBJECTID'
    ,fields          = flds
);

print("  tracking unmatched stations");
cb_nIfNone = """
def nIfNone(pin):
    if pin is None:
       return 'N';
    else:
       return 'Y';
""";

arcpy.management.CalculateField(
     in_table        = out_file
    ,field           = 'has_creat3_histstorm_data'
    ,expression      = "nIfNone(!IntensePrecip_Historic_5Year_24!)"
    ,expression_type = 'PYTHON3'
    ,code_block      = cb_nIfNone
);

print("  zeroing out unmatched stations");
cb_zeroIfNone = """
def zeroIfNone(pin):
    if pin is None:
       return 0;
    else:
       return pin;
""";

for fld in flds:

    arcpy.management.CalculateField(
         in_table        = out_file
        ,field           = fld
        ,expression      = "zeroIfNone(!" + fld + "!)"
        ,expression_type = 'PYTHON3'
        ,code_block      = cb_zeroIfNone
    );


  copying to new work stations
  add tracking fields for joined data
  calculating the nearest historic station
  updating Near fields for clarity
  adding indexes
  joining in historic storms information
  tracking unmatched stations
  zeroing out unmatched stations
Wall time: 11.3 s


### 60.20: Calculate the containing CREAT3 grid with PRECIP data for stations

In [24]:
%%time

in_file  = wrkfgdb + os.sep + 'D4EMLite_PREC_Details_WRK2';
out_file = wrkfgdb + os.sep + 'D4EMLite_PREC_Details_WRK3';

if arcpy.Exists(out_file):
    arcpy.Delete_management(out_file);
    
print("  calculating the containing CREAT3 grid with precipitation data");

flds = [
     'PRECIP_2035_HOTDRY_JAN_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_FEB_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_MAR_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_APR_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_MAY_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_JUN_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_JUL_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_AUG_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_SEP_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_OCT_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_NOV_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_DEC_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_ANNUAL_CHANG'

    ,'PRECIP_2035_CENTRAL_JAN_CHANGEP'
    ,'PRECIP_2035_CENTRAL_FEB_CHANGEP'
    ,'PRECIP_2035_CENTRAL_MAR_CHANGEP'
    ,'PRECIP_2035_CENTRAL_APR_CHANGEP'
    ,'PRECIP_2035_CENTRAL_MAY_CHANGEP'
    ,'PRECIP_2035_CENTRAL_JUN_CHANGEP'
    ,'PRECIP_2035_CENTRAL_JUL_CHANGEP'
    ,'PRECIP_2035_CENTRAL_AUG_CHANGEP'
    ,'PRECIP_2035_CENTRAL_SEP_CHANGEP'
    ,'PRECIP_2035_CENTRAL_OCT_CHANGEP'
    ,'PRECIP_2035_CENTRAL_NOV_CHANGEP'
    ,'PRECIP_2035_CENTRAL_DEC_CHANGEP'
    ,'PRECIP_2035_CENTRAL_ANNUAL_CHAN'

    ,'PRECIP_2035_WETWARM_JAN_CHANGEP'
    ,'PRECIP_2035_WETWARM_FEB_CHANGEP'
    ,'PRECIP_2035_WETWARM_MAR_CHANGEP'
    ,'PRECIP_2035_WETWARM_APR_CHANGEP'
    ,'PRECIP_2035_WETWARM_MAY_CHANGEP'
    ,'PRECIP_2035_WETWARM_JUN_CHANGEP'
    ,'PRECIP_2035_WETWARM_JUL_CHANGEP'
    ,'PRECIP_2035_WETWARM_AUG_CHANGEP'
    ,'PRECIP_2035_WETWARM_SEP_CHANGEP'
    ,'PRECIP_2035_WETWARM_OCT_CHANGEP'
    ,'PRECIP_2035_WETWARM_NOV_CHANGEP'
    ,'PRECIP_2035_WETWARM_DEC_CHANGEP'
    ,'PRECIP_2035_WETWARM_ANNUAL_CHAN'

    ,'PRECIP_2060_HOTDRY_JAN_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_FEB_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_MAR_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_APR_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_MAY_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_JUN_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_JUL_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_AUG_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_SEP_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_OCT_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_NOV_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_DEC_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_ANNUAL_CHANG'

    ,'PRECIP_2060_CENTRAL_JAN_CHANGEP'
    ,'PRECIP_2060_CENTRAL_FEB_CHANGEP'
    ,'PRECIP_2060_CENTRAL_MAR_CHANGEP'
    ,'PRECIP_2060_CENTRAL_APR_CHANGEP'
    ,'PRECIP_2060_CENTRAL_MAY_CHANGEP'
    ,'PRECIP_2060_CENTRAL_JUN_CHANGEP'
    ,'PRECIP_2060_CENTRAL_JUL_CHANGEP'
    ,'PRECIP_2060_CENTRAL_AUG_CHANGEP'
    ,'PRECIP_2060_CENTRAL_SEP_CHANGEP'
    ,'PRECIP_2060_CENTRAL_OCT_CHANGEP'
    ,'PRECIP_2060_CENTRAL_NOV_CHANGEP'
    ,'PRECIP_2060_CENTRAL_DEC_CHANGEP'
    ,'PRECIP_2060_CENTRAL_ANNUAL_CHAN'

    ,'PRECIP_2060_WETWARM_JAN_CHANGEP'
    ,'PRECIP_2060_WETWARM_FEB_CHANGEP'
    ,'PRECIP_2060_WETWARM_MAR_CHANGEP'
    ,'PRECIP_2060_WETWARM_APR_CHANGEP'
    ,'PRECIP_2060_WETWARM_MAY_CHANGEP'
    ,'PRECIP_2060_WETWARM_JUN_CHANGEP'
    ,'PRECIP_2060_WETWARM_JUL_CHANGEP'
    ,'PRECIP_2060_WETWARM_AUG_CHANGEP'
    ,'PRECIP_2060_WETWARM_SEP_CHANGEP'
    ,'PRECIP_2060_WETWARM_OCT_CHANGEP'
    ,'PRECIP_2060_WETWARM_NOV_CHANGEP'
    ,'PRECIP_2060_WETWARM_DEC_CHANGEP'
    ,'PRECIP_2060_WETWARM_ANNUAL_CHAN'

    ,'STORM5YR_2035_STORMY_ANNUAL_CHA'
    ,'STORM10YR_2035_STORMY_ANNUAL_CH'
    ,'STORM15YR_2035_STORMY_ANNUAL_CH'
    ,'STORM30YR_2035_STORMY_ANNUAL_CH'
    ,'STORM50YR_2035_STORMY_ANNUAL_CH'
    ,'STORM100YR_2035_STORMY_ANNUAL_C'
    
    ,'STORM5YR_2060_STORMY_ANNUAL_CHA'
    ,'STORM10YR_2060_STORMY_ANNUAL_CH'
    ,'STORM15YR_2060_STORMY_ANNUAL_CH'
    ,'STORM30YR_2060_STORMY_ANNUAL_CH'
    ,'STORM50YR_2060_STORMY_ANNUAL_CH'
    ,'STORM100YR_2060_STORMY_ANNUAL_C'
    
    ,'STORM5YR_2035_LESSSTORMY_ANNUAL'
    ,'STORM10YR_2035_LESSSTORMY_ANNUA'
    ,'STORM15YR_2035_LESSSTORMY_ANNUA'
    ,'STORM30YR_2035_LESSSTORMY_ANNUA'
    ,'STORM50YR_2035_LESSSTORMY_ANNUA'
    ,'STORM100YR_2035_LESSSTORMY_ANNU'
    
    ,'STORM5YR_2060_LESSSTORMY_ANNUAL'
    ,'STORM10YR_2060_LESSSTORMY_ANNUA'
    ,'STORM15YR_2060_LESSSTORMY_ANNUA'
    ,'STORM30YR_2060_LESSSTORMY_ANNUA'
    ,'STORM50YR_2060_LESSSTORMY_ANNUA'
    ,'STORM100YR_2060_LESSSTORMY_ANNU'
];
  
arcpy.analysis.SpatialJoin(
     target_features   = in_file
    ,join_features     = grid
    ,out_feature_class = out_file
    ,join_operation    = 'JOIN_ONE_TO_ONE'
    ,join_type         = 'KEEP_ALL'
    ,match_option      = 'WITHIN'
);

print("  updating tracking fields for grid source");
arcpy.management.CalculateField(
     in_table        = out_file
    ,field           = 'has_creat3_grid_data'
    ,expression      = "nIfNone(!PRECIP_2035_HOTDRY_JAN_CHANGEPC!)"
    ,expression_type = 'PYTHON3'
    ,code_block      = cb_nIfNone
);

arcpy.management.CalculateField(
     in_table        = out_file
    ,field           = 'is_interp_grid_data'
    ,expression      = "!is_interpolated!"
    ,expression_type = 'PYTHON3'
);

print("  cleaning up unneeded fields")
arcpy.management.DeleteField(
     in_table   = out_file
    ,drop_field = [
         'join_count'
        ,'TARGET_FID'
        ,'NEAR_FID_HIST'
        ,'NEAR_DIST_HIST'
        ,'CREAT_ID'
        ,'GRIDCODE'
        ,'HurrGridID'
        ,'grid_over_land'
        ,'Longitude_Centroid'
        ,'Latitude_Centroid'
        ,'Longitude_Min'
        ,'Latitude_Min'
        ,'Longitude_Max'
        ,'Latitude_Max'
        ,'TEMP_2035_HOTDRY_JAN_CHANGEF'
        ,'TEMP_2035_HOTDRY_FEB_CHANGEF'
        ,'TEMP_2035_HOTDRY_MAR_CHANGEF'
        ,'TEMP_2035_HOTDRY_APR_CHANGEF'
        ,'TEMP_2035_HOTDRY_MAY_CHANGEF'
        ,'TEMP_2035_HOTDRY_JUN_CHANGEF'
        ,'TEMP_2035_HOTDRY_JUL_CHANGEF'
        ,'TEMP_2035_HOTDRY_AUG_CHANGEF'
        ,'TEMP_2035_HOTDRY_SEP_CHANGEF'
        ,'TEMP_2035_HOTDRY_OCT_CHANGEF'
        ,'TEMP_2035_HOTDRY_NOV_CHANGEF'
        ,'TEMP_2035_HOTDRY_DEC_CHANGEF'
        ,'TEMP_2035_HOTDRY_ANNUAL_CHANGEF'
        ,'TEMP_2035_CENTRAL_JAN_CHANGEF'
        ,'TEMP_2035_CENTRAL_FEB_CHANGEF'
        ,'TEMP_2035_CENTRAL_MAR_CHANGEF'
        ,'TEMP_2035_CENTRAL_APR_CHANGEF'
        ,'TEMP_2035_CENTRAL_MAY_CHANGEF'
        ,'TEMP_2035_CENTRAL_JUN_CHANGEF'
        ,'TEMP_2035_CENTRAL_JUL_CHANGEF'
        ,'TEMP_2035_CENTRAL_AUG_CHANGEF'
        ,'TEMP_2035_CENTRAL_SEP_CHANGEF'
        ,'TEMP_2035_CENTRAL_OCT_CHANGEF'
        ,'TEMP_2035_CENTRAL_NOV_CHANGEF'
        ,'TEMP_2035_CENTRAL_DEC_CHANGEF'
        ,'TEMP_2035_CENTRAL_ANNUAL_CHANGE'
        ,'TEMP_2035_WETWARM_JAN_CHANGEF'
        ,'TEMP_2035_WETWARM_FEB_CHANGEF'
        ,'TEMP_2035_WETWARM_MAR_CHANGEF'
        ,'TEMP_2035_WETWARM_APR_CHANGEF'
        ,'TEMP_2035_WETWARM_MAY_CHANGEF'
        ,'TEMP_2035_WETWARM_JUN_CHANGEF'
        ,'TEMP_2035_WETWARM_JUL_CHANGEF'
        ,'TEMP_2035_WETWARM_AUG_CHANGEF'
        ,'TEMP_2035_WETWARM_SEP_CHANGEF'
        ,'TEMP_2035_WETWARM_OCT_CHANGEF'
        ,'TEMP_2035_WETWARM_NOV_CHANGEF'
        ,'TEMP_2035_WETWARM_DEC_CHANGEF'
        ,'TEMP_2035_WETWARM_ANNUAL_CHANGE'
        ,'TEMP_2060_HOTDRY_JAN_CHANGEF'
        ,'TEMP_2060_HOTDRY_FEB_CHANGEF'
        ,'TEMP_2060_HOTDRY_MAR_CHANGEF'
        ,'TEMP_2060_HOTDRY_APR_CHANGEF'
        ,'TEMP_2060_HOTDRY_MAY_CHANGEF'
        ,'TEMP_2060_HOTDRY_JUN_CHANGEF'
        ,'TEMP_2060_HOTDRY_JUL_CHANGEF'
        ,'TEMP_2060_HOTDRY_AUG_CHANGEF'
        ,'TEMP_2060_HOTDRY_SEP_CHANGEF'
        ,'TEMP_2060_HOTDRY_OCT_CHANGEF'
        ,'TEMP_2060_HOTDRY_NOV_CHANGEF'
        ,'TEMP_2060_HOTDRY_DEC_CHANGEF'
        ,'TEMP_2060_HOTDRY_ANNUAL_CHANGEF'
        ,'TEMP_2060_CENTRAL_JAN_CHANGEF'
        ,'TEMP_2060_CENTRAL_FEB_CHANGEF'
        ,'TEMP_2060_CENTRAL_MAR_CHANGEF'
        ,'TEMP_2060_CENTRAL_APR_CHANGEF'
        ,'TEMP_2060_CENTRAL_MAY_CHANGEF'
        ,'TEMP_2060_CENTRAL_JUN_CHANGEF'
        ,'TEMP_2060_CENTRAL_JUL_CHANGEF'
        ,'TEMP_2060_CENTRAL_AUG_CHANGEF'
        ,'TEMP_2060_CENTRAL_SEP_CHANGEF'
        ,'TEMP_2060_CENTRAL_OCT_CHANGEF'
        ,'TEMP_2060_CENTRAL_NOV_CHANGEF'
        ,'TEMP_2060_CENTRAL_DEC_CHANGEF'
        ,'TEMP_2060_CENTRAL_ANNUAL_CHANGE'
        ,'TEMP_2060_WETWARM_JAN_CHANGEF'
        ,'TEMP_2060_WETWARM_FEB_CHANGEF'
        ,'TEMP_2060_WETWARM_MAR_CHANGEF'
        ,'TEMP_2060_WETWARM_APR_CHANGEF'
        ,'TEMP_2060_WETWARM_MAY_CHANGEF'
        ,'TEMP_2060_WETWARM_JUN_CHANGEF'
        ,'TEMP_2060_WETWARM_JUL_CHANGEF'
        ,'TEMP_2060_WETWARM_AUG_CHANGEF'
        ,'TEMP_2060_WETWARM_SEP_CHANGEF'
        ,'TEMP_2060_WETWARM_OCT_CHANGEF'
        ,'TEMP_2060_WETWARM_NOV_CHANGEF'
        ,'TEMP_2060_WETWARM_DEC_CHANGEF'
        ,'TEMP_2060_WETWARM_ANNUAL_CHANGE'
        ,'is_interpolated'
    ]
);

print("  zeroing out unmatched stations");
for fld in flds:
   
    arcpy.management.CalculateField(
         in_table        = out_file
        ,field           = fld
        ,expression      = "zeroIfNone(!" + fld + "!)"
        ,expression_type = 'PYTHON3'
        ,code_block      = cb_zeroIfNone
    );
    

  calculating the containing CREAT3 grid with precipitation data
  updating tracking fields for grid source
  cleaning up unneeded fields
  zeroing out unmatched stations
Wall time: 1min 46s


### 60.30: Determine Precipitation RF value for stations

In [25]:
%%time

in_file  = wrkfgdb + os.sep + 'D4EMLite_PREC_Details_WRK3';
out_file = wrkfgdb + os.sep + 'D4EMLite_PREC_Details_WRK4';

if arcpy.Exists(out_file):
    arcpy.Delete_management(out_file);

print("  building spatial join with precipitation coverage");
arcpy.analysis.SpatialJoin(
     target_features   = in_file
    ,join_features     = rfcov
    ,out_feature_class = out_file
    ,join_operation    = 'JOIN_ONE_TO_ONE'
    ,join_type         = 'KEEP_ALL'
    ,match_option      = 'INTERSECT'
);

print("  tracking unmatched stations");
z = arcpy.management.CalculateField(
     in_table        = out_file
    ,field           = 'has_rainfall_data'
    ,expression      = "nIfNone(!rf_value!)"
    ,expression_type = 'PYTHON3'
    ,code_block      = cb_nIfNone
);


  building spatial join with precipitation coverage
  tracking unmatched stations
Wall time: 21.1 s


### 60.40: Generate tables for PREC extracts 

In [26]:
%%time

def createPREC(fgdb,fc):
    
    if arcpy.Exists(fgdb + os.sep + fc):
        arcpy.Delete_management(fgdb + os.sep + fc);
    
    arcpy.CreateFeatureclass_management(
         out_path      = fgdb
        ,out_name      = fc
        ,geometry_type = "POINT"
        ,has_m         = "DISABLED"
        ,has_z         = "DISABLED"
        ,spatial_reference = arcpy.SpatialReference(4269) 
    );
    
    arcpy.management.AddFields(
         in_table = fgdb + os.sep + fc
        ,field_description = [
             ['StationId','TEXT'  ,'StationId',14]
            ,['Jan'      ,'DOUBLE','Jan']
            ,['Feb'      ,'DOUBLE','Feb']
            ,['Mar'      ,'DOUBLE','Mar']
            ,['Apr'      ,'DOUBLE','Apr']
            ,['May'      ,'DOUBLE','May']
            ,['Jun'      ,'DOUBLE','Jun']
            ,['Jul'      ,'DOUBLE','Jul']
            ,['Aug'      ,'DOUBLE','Aug']
            ,['Sep'      ,'DOUBLE','Sep']
            ,['Oct'      ,'DOUBLE','Oct']
            ,['Nov'      ,'DOUBLE','Nov']
            ,['Dec'      ,'DOUBLE','Dec']
            ,['Ann'      ,'DOUBLE','Ann']
         ]
    );
    
def createGEVdepth(fgdb,fc):
    
    if arcpy.Exists(fgdb + os.sep + fc):
        arcpy.Delete_management(fgdb + os.sep + fc);
    
    arcpy.CreateFeatureclass_management(
         out_path      = fgdb
        ,out_name      = fc
        ,geometry_type = "POINT"
        ,has_m         = "DISABLED"
        ,has_z         = "DISABLED"
        ,spatial_reference = arcpy.SpatialReference(4269) 
    );
    
    arcpy.management.AddFields(
         in_table = fgdb + os.sep + fc
        ,field_description = [
             ['StationId'   ,'TEXT'  ,'StationId',14]
            ,['z5yrann_mm'  ,'DOUBLE','5yrann_mm']
            ,['z10yrann_mm' ,'DOUBLE','10yrann_mm']
            ,['z15yrann_mm' ,'DOUBLE','15yrann_mm']
            ,['z30yrann_mm' ,'DOUBLE','30yrann_mm']
            ,['z50yrann_mm' ,'DOUBLE','50yrann_mm']
            ,['z100yrann_mm','DOUBLE','100yrann_mm']
         ]
    );
 
print("  creating fresh PREC feature classes");
createPREC(rezfgdb,'PREC2035HotDry');
createPREC(rezfgdb,'PREC2035Central');
createPREC(rezfgdb,'PREC2035WetWarm');
createPREC(rezfgdb,'PREC2060HotDry');
createPREC(rezfgdb,'PREC2060Central');
createPREC(rezfgdb,'PREC2060WetWarm');

print("  creating fresh GEVddepth feature classes");
createGEVdepth(rezfgdb,'GEVdepth2035Stormy');
createGEVdepth(rezfgdb,'GEVdepth2035LessStormy');
createGEVdepth(rezfgdb,'GEVdepth2060Stormy');
createGEVdepth(rezfgdb,'GEVdepth2060LessStormy');
createGEVdepth(rezfgdb,'GEVdepth_historical');

print("  creating fresh PREC_SCS_Types feature class");
if arcpy.Exists(rezfgdb + os.sep + 'PREC_SCS_Types'):
    arcpy.Delete_management(rezfgdb + os.sep + 'PREC_SCS_Types');
        
arcpy.CreateFeatureclass_management(
     out_path      = rezfgdb
    ,out_name      = 'PREC_SCS_Types'
    ,geometry_type = "POINT"
    ,has_m         = "DISABLED"
    ,has_z         = "DISABLED"
    ,spatial_reference = arcpy.SpatialReference(4269) 
);

z = arcpy.management.AddFields(
     in_table = rezfgdb + os.sep + 'PREC_SCS_Types'
    ,field_description = [
         ['StationId'                 ,'TEXT','StationId',14]
        ,['Rainfall_Distribution_Type','TEXT','Rainfall_Distribution_Type',255]
     ]
);


  creating fresh PREC feature classes
  creating fresh GEVddepth feature classes
  creating fresh PREC_SCS_Types feature class
Wall time: 19.8 s


### 60.50: Write out datasets to individual feature classes

In [27]:
%%time

in_file = wrkfgdb + os.sep + 'D4EMLite_PREC_Details_WRK4';

fldin = [
     'StationId'
    # 1
    ,'PRECIP_2035_HOTDRY_JAN_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_FEB_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_MAR_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_APR_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_MAY_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_JUN_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_JUL_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_AUG_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_SEP_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_OCT_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_NOV_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_DEC_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_ANNUAL_CHANG'
    # 14
    ,'PRECIP_2035_CENTRAL_JAN_CHANGEP'
    ,'PRECIP_2035_CENTRAL_FEB_CHANGEP'
    ,'PRECIP_2035_CENTRAL_MAR_CHANGEP'
    ,'PRECIP_2035_CENTRAL_APR_CHANGEP'
    ,'PRECIP_2035_CENTRAL_MAY_CHANGEP'
    ,'PRECIP_2035_CENTRAL_JUN_CHANGEP'
    ,'PRECIP_2035_CENTRAL_JUL_CHANGEP'
    ,'PRECIP_2035_CENTRAL_AUG_CHANGEP'
    ,'PRECIP_2035_CENTRAL_SEP_CHANGEP'
    ,'PRECIP_2035_CENTRAL_OCT_CHANGEP'
    ,'PRECIP_2035_CENTRAL_NOV_CHANGEP'
    ,'PRECIP_2035_CENTRAL_DEC_CHANGEP'
    ,'PRECIP_2035_CENTRAL_ANNUAL_CHAN'
    # 27
    ,'PRECIP_2035_WETWARM_JAN_CHANGEP'
    ,'PRECIP_2035_WETWARM_FEB_CHANGEP'
    ,'PRECIP_2035_WETWARM_MAR_CHANGEP'
    ,'PRECIP_2035_WETWARM_APR_CHANGEP'
    ,'PRECIP_2035_WETWARM_MAY_CHANGEP'
    ,'PRECIP_2035_WETWARM_JUN_CHANGEP'
    ,'PRECIP_2035_WETWARM_JUL_CHANGEP'
    ,'PRECIP_2035_WETWARM_AUG_CHANGEP'
    ,'PRECIP_2035_WETWARM_SEP_CHANGEP'
    ,'PRECIP_2035_WETWARM_OCT_CHANGEP'
    ,'PRECIP_2035_WETWARM_NOV_CHANGEP'
    ,'PRECIP_2035_WETWARM_DEC_CHANGEP'
    ,'PRECIP_2035_WETWARM_ANNUAL_CHAN'
    # 40
    ,'PRECIP_2060_HOTDRY_JAN_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_FEB_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_MAR_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_APR_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_MAY_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_JUN_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_JUL_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_AUG_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_SEP_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_OCT_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_NOV_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_DEC_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_ANNUAL_CHANG'
    # 53
    ,'PRECIP_2060_CENTRAL_JAN_CHANGEP'
    ,'PRECIP_2060_CENTRAL_FEB_CHANGEP'
    ,'PRECIP_2060_CENTRAL_MAR_CHANGEP'
    ,'PRECIP_2060_CENTRAL_APR_CHANGEP'
    ,'PRECIP_2060_CENTRAL_MAY_CHANGEP'
    ,'PRECIP_2060_CENTRAL_JUN_CHANGEP'
    ,'PRECIP_2060_CENTRAL_JUL_CHANGEP'
    ,'PRECIP_2060_CENTRAL_AUG_CHANGEP'
    ,'PRECIP_2060_CENTRAL_SEP_CHANGEP'
    ,'PRECIP_2060_CENTRAL_OCT_CHANGEP'
    ,'PRECIP_2060_CENTRAL_NOV_CHANGEP'
    ,'PRECIP_2060_CENTRAL_DEC_CHANGEP'
    ,'PRECIP_2060_CENTRAL_ANNUAL_CHAN'
    # 66 
    ,'PRECIP_2060_WETWARM_JAN_CHANGEP'
    ,'PRECIP_2060_WETWARM_FEB_CHANGEP'
    ,'PRECIP_2060_WETWARM_MAR_CHANGEP'
    ,'PRECIP_2060_WETWARM_APR_CHANGEP'
    ,'PRECIP_2060_WETWARM_MAY_CHANGEP'
    ,'PRECIP_2060_WETWARM_JUN_CHANGEP'
    ,'PRECIP_2060_WETWARM_JUL_CHANGEP'
    ,'PRECIP_2060_WETWARM_AUG_CHANGEP'
    ,'PRECIP_2060_WETWARM_SEP_CHANGEP'
    ,'PRECIP_2060_WETWARM_OCT_CHANGEP'
    ,'PRECIP_2060_WETWARM_NOV_CHANGEP'
    ,'PRECIP_2060_WETWARM_DEC_CHANGEP'
    ,'PRECIP_2060_WETWARM_ANNUAL_CHAN'
    # 79
    ,'STORM5YR_2035_STORMY_ANNUAL_CHA'
    ,'STORM10YR_2035_STORMY_ANNUAL_CH'
    ,'STORM15YR_2035_STORMY_ANNUAL_CH'
    ,'STORM30YR_2035_STORMY_ANNUAL_CH'
    ,'STORM50YR_2035_STORMY_ANNUAL_CH'
    ,'STORM100YR_2035_STORMY_ANNUAL_C'
    # 85
    ,'STORM5YR_2060_STORMY_ANNUAL_CHA'
    ,'STORM10YR_2060_STORMY_ANNUAL_CH'
    ,'STORM15YR_2060_STORMY_ANNUAL_CH'
    ,'STORM30YR_2060_STORMY_ANNUAL_CH'
    ,'STORM50YR_2060_STORMY_ANNUAL_CH'
    ,'STORM100YR_2060_STORMY_ANNUAL_C'
    # 91
    ,'STORM5YR_2035_LESSSTORMY_ANNUAL'
    ,'STORM10YR_2035_LESSSTORMY_ANNUA'
    ,'STORM15YR_2035_LESSSTORMY_ANNUA'
    ,'STORM30YR_2035_LESSSTORMY_ANNUA'
    ,'STORM50YR_2035_LESSSTORMY_ANNUA'
    ,'STORM100YR_2035_LESSSTORMY_ANNU'
    # 97
    ,'STORM5YR_2060_LESSSTORMY_ANNUAL'
    ,'STORM10YR_2060_LESSSTORMY_ANNUA'
    ,'STORM15YR_2060_LESSSTORMY_ANNUA'
    ,'STORM30YR_2060_LESSSTORMY_ANNUA'
    ,'STORM50YR_2060_LESSSTORMY_ANNUA'
    ,'STORM100YR_2060_LESSSTORMY_ANNU'
    # 103
    ,'IntensePrecip_Historic_5Year_24'
    ,'IntensePrecip_Historic_10Year_2'
    ,'IntensePrecip_Historic_15Year_2'
    ,'IntensePrecip_Historic_30Year_2'
    ,'IntensePrecip_Historic_50Year_2'
    ,'IntensePrecip_Historic_100Year_'
    # 109
    ,'rf_value'
];

fldprec = [
     'StationId'
    ,'Jan'
    ,'Feb'
    ,'Mar'
    ,'Apr'
    ,'May'
    ,'Jun'
    ,'Jul'
    ,'Aug'
    ,'Sep'
    ,'Oct'
    ,'Nov'
    ,'Dec'
    ,'Ann'
];

fldgev = [
     'StationId'
    ,'z5yrann_mm'
    ,'z10yrann_mm'
    ,'z15yrann_mm'
    ,'z30yrann_mm'
    ,'z50yrann_mm'
    ,'z100yrann_mm'
];

fldscs = [
     'StationId'
    ,'Rainfall_Distribution_Type'
]

print("  truncate datasets for safety");
arcpy.TruncateTable_management(rezfgdb + os.sep + 'PREC2035HotDry');
arcpy.TruncateTable_management(rezfgdb + os.sep + 'PREC2035Central');
arcpy.TruncateTable_management(rezfgdb + os.sep + 'PREC2035WetWarm');
arcpy.TruncateTable_management(rezfgdb + os.sep + 'PREC2060HotDry');
arcpy.TruncateTable_management(rezfgdb + os.sep + 'PREC2060Central');
arcpy.TruncateTable_management(rezfgdb + os.sep + 'PREC2060WetWarm');
arcpy.TruncateTable_management(rezfgdb + os.sep + 'GEVdepth2035Stormy');
arcpy.TruncateTable_management(rezfgdb + os.sep + 'GEVdepth2035LessStormy');
arcpy.TruncateTable_management(rezfgdb + os.sep + 'GEVdepth2060Stormy');
arcpy.TruncateTable_management(rezfgdb + os.sep + 'GEVdepth2060LessStormy');
arcpy.TruncateTable_management(rezfgdb + os.sep + 'GEVdepth_historical');
arcpy.TruncateTable_management(rezfgdb + os.sep + 'PREC_SCS_Types');

print("  opening an editor session to support multiple cursors");
with arcpy.da.Editor(rezfgdb) as edit:
      
    curs_PREC2035HotDry = arcpy.da.InsertCursor(
         in_table    = rezfgdb + os.sep + 'PREC2035HotDry'   
        ,field_names = fldprec
    );
    
    curs_PREC2035Central = arcpy.da.InsertCursor(
         in_table    = rezfgdb + os.sep + 'PREC2035Central'   
        ,field_names = fldprec
    );
        
    curs_PREC2035WetWarm = arcpy.da.InsertCursor(
         in_table    = rezfgdb + os.sep + 'PREC2035WetWarm'   
        ,field_names = fldprec
    );
    
    curs_PREC2060HotDry = arcpy.da.InsertCursor(
         in_table    = rezfgdb + os.sep + 'PREC2060HotDry'   
        ,field_names = fldprec
    );
    
    curs_PREC2060Central = arcpy.da.InsertCursor(
         in_table    = rezfgdb + os.sep + 'PREC2060Central'   
        ,field_names = fldprec
    );
        
    curs_PREC2060WetWarm = arcpy.da.InsertCursor(
         in_table    = rezfgdb + os.sep + 'PREC2060WetWarm'   
        ,field_names = fldprec
    );
    
    curs_GEVdepth2035Stormy = arcpy.da.InsertCursor(
         in_table    = rezfgdb + os.sep + 'GEVdepth2035Stormy'   
        ,field_names = fldgev
    );
        
    curs_GEVdepth2035LessStormy = arcpy.da.InsertCursor(
         in_table    = rezfgdb + os.sep + 'GEVdepth2035LessStormy'   
        ,field_names = fldgev
    );
    
    curs_GEVdepth2060Stormy = arcpy.da.InsertCursor(
         in_table    = rezfgdb + os.sep + 'GEVdepth2060Stormy'   
        ,field_names = fldgev
    );
        
    curs_GEVdepth2060LessStormy = arcpy.da.InsertCursor(
         in_table    = rezfgdb + os.sep + 'GEVdepth2060LessStormy'   
        ,field_names = fldgev
    );
    
    curs_GEVdepth_historical = arcpy.da.InsertCursor(
         in_table    = rezfgdb + os.sep + 'GEVdepth_historical'   
        ,field_names = fldgev
    );
    
    curs_PREC_SCS_Types = arcpy.da.InsertCursor(
         in_table    = rezfgdb + os.sep + 'PREC_SCS_Types'   
        ,field_names = fldscs
    );

    print("  writing the station data to individual feature classes");
    with arcpy.da.SearchCursor(in_file,fldin) as incur:
    
        for row in incur:
            
            # PRECIP 2035 HOTDRY
            curs_PREC2035HotDry.insertRow((
                 row[0]
                ,row[1]
                ,row[2]
                ,row[3]
                ,row[4]
                ,row[5]
                ,row[6]
                ,row[7]
                ,row[8]
                ,row[9]
                ,row[10]
                ,row[11]
                ,row[12]
                ,row[13]
            ));
            
            # PRECIP 2035 CENTRAL
            curs_PREC2035Central.insertRow((
                 row[0]
                ,row[14]
                ,row[15]
                ,row[16]
                ,row[17]
                ,row[18]
                ,row[19]
                ,row[20]
                ,row[21]
                ,row[22]
                ,row[23]
                ,row[24]
                ,row[25]
                ,row[26]
            ));
            
            # PRECIP 2035 WETWARM
            curs_PREC2035WetWarm.insertRow((
                 row[0]
                ,row[27]
                ,row[28]
                ,row[29]
                ,row[30]
                ,row[31]
                ,row[32]
                ,row[33]
                ,row[34]
                ,row[35]
                ,row[36]
                ,row[37]
                ,row[38]
                ,row[39]
            ));
            
            # PRECIP 2060 HOTDRY
            curs_PREC2060HotDry.insertRow((
                 row[0]
                ,row[40]
                ,row[41]
                ,row[42]
                ,row[43]
                ,row[44]
                ,row[45]
                ,row[46]
                ,row[47]
                ,row[48]
                ,row[49]
                ,row[50]
                ,row[51]
                ,row[52]
            ));
            
            # PRECIP 2060 CENTRAL
            curs_PREC2060Central.insertRow((
                 row[0]
                ,row[53]
                ,row[54]
                ,row[55]
                ,row[56]
                ,row[57]
                ,row[58]
                ,row[59]
                ,row[60]
                ,row[61]
                ,row[62]
                ,row[63]
                ,row[64]
                ,row[65]
            ));
            
            # PRECIP 2060 WETWARM
            curs_PREC2060WetWarm.insertRow((
                 row[0]
                ,row[66]
                ,row[67]
                ,row[68]
                ,row[69]
                ,row[70]
                ,row[71]
                ,row[72]
                ,row[73]
                ,row[74]
                ,row[75]
                ,row[76]
                ,row[77]
                ,row[78]
            ));
            
            IntensePrecip_Historic_5Year_mm   = row[103] * 25.4;
            IntensePrecip_Historic_10Year_mm  = row[104] * 25.4;
            IntensePrecip_Historic_15Year_mm  = row[105] * 25.4;
            IntensePrecip_Historic_30Year_mm  = row[106] * 25.4;
            IntensePrecip_Historic_50Year_mm  = row[107] * 25.4;
            IntensePrecip_Historic_100Year_mm = row[108] * 25.4;
            
            # STORMY 2035
            curs_GEVdepth2035Stormy.insertRow((
                 row[0]
                ,IntensePrecip_Historic_5Year_mm   * (1 + row[79]  / 100)
                ,IntensePrecip_Historic_10Year_mm  * (1 + row[80]  / 100)
                ,IntensePrecip_Historic_15Year_mm  * (1 + row[81]  / 100)
                ,IntensePrecip_Historic_30Year_mm  * (1 + row[82]  / 100)
                ,IntensePrecip_Historic_50Year_mm  * (1 + row[83]  / 100)
                ,IntensePrecip_Historic_100Year_mm * (1 + row[84]  / 100)
            ));
            
            # STORMY 2060
            curs_GEVdepth2060Stormy.insertRow((
                 row[0]
                ,IntensePrecip_Historic_5Year_mm   * (1 + row[85]  / 100)
                ,IntensePrecip_Historic_10Year_mm  * (1 + row[86]  / 100)
                ,IntensePrecip_Historic_15Year_mm  * (1 + row[87]  / 100)
                ,IntensePrecip_Historic_30Year_mm  * (1 + row[88]  / 100)
                ,IntensePrecip_Historic_50Year_mm  * (1 + row[89]  / 100)
                ,IntensePrecip_Historic_100Year_mm * (1 + row[90]  / 100) 
            ));
            
            # LESS STORMY 2035
            curs_GEVdepth2035LessStormy.insertRow((
                 row[0]
                ,IntensePrecip_Historic_5Year_mm   * (1 + row[91]  / 100)
                ,IntensePrecip_Historic_10Year_mm  * (1 + row[92]  / 100)
                ,IntensePrecip_Historic_15Year_mm  * (1 + row[93]  / 100)
                ,IntensePrecip_Historic_30Year_mm  * (1 + row[94]  / 100)
                ,IntensePrecip_Historic_50Year_mm  * (1 + row[95]  / 100)
                ,IntensePrecip_Historic_100Year_mm * (1 + row[96]  / 100)
            ));
            
            # LESS STORMY 2060
            curs_GEVdepth2060LessStormy.insertRow((
                 row[0]
                ,IntensePrecip_Historic_5Year_mm   * (1 + row[97]  / 100)
                ,IntensePrecip_Historic_10Year_mm  * (1 + row[98]  / 100)
                ,IntensePrecip_Historic_15Year_mm  * (1 + row[99]  / 100)
                ,IntensePrecip_Historic_30Year_mm  * (1 + row[100] / 100)
                ,IntensePrecip_Historic_50Year_mm  * (1 + row[101] / 100)
                ,IntensePrecip_Historic_100Year_mm * (1 + row[102] / 100)
            ));
            
            # Intense Precipitation Historic in mm
            curs_GEVdepth_historical.insertRow((
                 row[0]
                ,IntensePrecip_Historic_5Year_mm
                ,IntensePrecip_Historic_10Year_mm
                ,IntensePrecip_Historic_15Year_mm
                ,IntensePrecip_Historic_30Year_mm
                ,IntensePrecip_Historic_50Year_mm
                ,IntensePrecip_Historic_100Year_mm
            ));
            
            curs_PREC_SCS_Types.insertRow((
                 row[0]
                ,row[109]
            ));


  truncate datasets for safety
  opening an editor session to support multiple cursors
  writing the station data to individual feature classes
Wall time: 9.6 s


### 60.60: Final Summary

In [28]:
%%time

in_file = wrkfgdb + os.sep + 'D4EMLite_PREC_Details_WRK4';

stat = rezfgdb + os.sep + 'D4EMLite_PREC_Details';

if arcpy.Exists(stat):
    arcpy.Delete_management(stat);
   
print("  copying to workstations results");
arcpy.conversion.FeatureClassToFeatureClass(
     in_features = in_file
    ,out_path    = rezfgdb
    ,out_name    = 'D4EMLite_PREC_Details'
);

print("  cleaning up the fieldset");
flds_to_keep = [
     'StationId'
    ,'DataType'
    ,'FileName'
    ,'Id'
    ,'Lat'
    ,'Long'
    ,'Scenario'
    ,'Constituent'
    ,'SDate'
    ,'EDate'
    ,'YrCount'
    ,'Value'
    ,'StaNam'
    ,'has_creat3_grid_data'
    ,'is_interp_grid_data'
    ,'has_creat3_histstorm_data'
    ,'has_rainfall_data'
    ,'Shape'
    ,'OBJECTID'
];

flds = [f.name for f in arcpy.ListFields(stat)]
delete_flds = list(set(flds) - set(flds_to_keep));

z = arcpy.management.DeleteField(
     in_table   = stat
    ,drop_field = delete_flds
);


  copying to workstations results
  cleaning up the fieldset
Wall time: 34.9 s


In [29]:
D4EMLite_PREC_Details_cnt  = arcpy.GetCount_management(rezfgdb + os.sep + 'D4EMLite_PREC_Details')[0];

PREC2035HotDry_cnt         = arcpy.GetCount_management(rezfgdb + os.sep + 'PREC2035HotDry')[0];
PREC2035Central_cnt        = arcpy.GetCount_management(rezfgdb + os.sep + 'PREC2035Central')[0];
PREC2035WetWarm_cnt        = arcpy.GetCount_management(rezfgdb + os.sep + 'PREC2035WetWarm')[0];

PREC2060HotDry_cnt         = arcpy.GetCount_management(rezfgdb + os.sep + 'PREC2060HotDry')[0];
PREC2060Central_cnt        = arcpy.GetCount_management(rezfgdb + os.sep + 'PREC2060Central')[0];
PREC2060WetWarm_cnt        = arcpy.GetCount_management(rezfgdb + os.sep + 'PREC2060WetWarm')[0];

GEVdepth2035Stormy_cnt     = arcpy.GetCount_management(rezfgdb + os.sep + 'GEVdepth2035Stormy')[0];
GEVdepth2035LessStormy_cnt = arcpy.GetCount_management(rezfgdb + os.sep + 'GEVdepth2035LessStormy')[0];

GEVdepth2060Stormy_cnt     = arcpy.GetCount_management(rezfgdb + os.sep + 'GEVdepth2060Stormy')[0];
GEVdepth2060LessStormy_cnt = arcpy.GetCount_management(rezfgdb + os.sep + 'GEVdepth2060LessStormy')[0];
GEVdepth_historical_cnt    = arcpy.GetCount_management(rezfgdb + os.sep + 'GEVdepth_historical')[0];

PREC_SCS_Types_cnt         = arcpy.GetCount_management(rezfgdb + os.sep + 'PREC_SCS_Types')[0];

print("  D4EMLite_PREC_Details Count : " + str(D4EMLite_PREC_Details_cnt));

print("  PREC2035HotDry Count        : " + str(PREC2035HotDry_cnt));
print("  PREC2035Central Count       : " + str(PREC2035Central_cnt));
print("  PREC2035WetWarm Count       : " + str(PREC2035WetWarm_cnt));

print("  PREC2060HotDry Count        : " + str(PREC2060HotDry_cnt));
print("  PREC2060Central Count       : " + str(PREC2060Central_cnt));
print("  PREC2060WetWarm Count       : " + str(PREC2060WetWarm_cnt));

print("  GEVdepth2035Stormy Count    : " + str(GEVdepth2035Stormy_cnt));
print("  GEVdepth2035LessStormy Count: " + str(GEVdepth2035LessStormy_cnt));

print("  GEVdepth2060Stormy Count    : " + str(GEVdepth2060Stormy_cnt));
print("  GEVdepth2060LessStormy Count: " + str(GEVdepth2060LessStormy_cnt));

print("  GEVdepth_historical Count   : " + str(GEVdepth_historical_cnt));

print("  PREC_SCS_Types Count        : " + str(PREC_SCS_Types_cnt));
print(" ");


  D4EMLite_PREC_Details Count : 5144
  PREC2035HotDry Count        : 5144
  PREC2035Central Count       : 5144
  PREC2035WetWarm Count       : 5144
  PREC2060HotDry Count        : 5144
  PREC2060Central Count       : 5144
  PREC2060WetWarm Count       : 5144
  GEVdepth2035Stormy Count    : 5144
  GEVdepth2035LessStormy Count: 5144
  GEVdepth2060Stormy Count    : 5144
  GEVdepth2060LessStormy Count: 5144
  GEVdepth_historical Count   : 5144
  PREC_SCS_Types Count        : 5144
 
